In [1]:
import pandas as pd 
import json 

In [2]:
with open('../data/annotation/output_frm_doccano/output_task_2/brit.jsonl', 'rb') as file:
    jsonl = file.read().splitlines()

In [49]:
def gen_idx_map(text):

    start_idx = 0
    idx_map = {}
    
    for idx, i in enumerate(text.split('\n')):
        
        end_idx = start_idx + len(i)
        idx_map[idx] = (start_idx, end_idx)
        start_idx = end_idx
    return idx_map

In [50]:
def add_labels_to_text(text, labels):

    idx_map = gen_idx_map(text)
    splat_text = text.split('\n')
    for label in labels:
        start_idx, end_idx ,label = label
        for idx, (s, e) in idx_map.items():
            if end_idx <= e:
                splat_text[idx] += ' - ' + label
                break
    return splat_text 

In [53]:
def process_transcripts(jsonl):

    transcripts = []
    for idx, line in enumerate(jsonl):
        j = json.loads(line)
        text = j['text']
        labels = j['label']
        if len(labels) < 1:
            break
        transcripts.append(add_labels_to_text(text, labels))
        
    return transcripts

In [122]:
def process_line(line):

    split_line = line.split(' - ')
    
    if len(split_line) > 2:
        text, label = split_line[0], split_line[1:]
        
    elif len(split_line) < 2:
        text, label = line, ['8-NO-PERSUASION']
        
    else:
        text, label = line.split(' - ')


    return text[10:], label

In [131]:
import re 

label_columns = [
    '1-RAPPORT',
    '2-NEGOTIATE',
    '3-EMOTION',
    '4-LOGIC',
    '5-AUTHORITY',
    '6-SOCIAL',
    '7-PRESSURE',
    '8-NO-PERSUASION'
]

def update_labels(label_columns, data, labels:list):
    if not isinstance(labels, list):
        labels = [labels]
        
    for i in label_columns:
        if i in labels:
            data[i].append(1)
        else:
            data[i].append(0)
    return data
    
def generate_dataframe(jsonl, label_columns):

    transcripts = process_transcripts(jsonl)
    
    data = {i : [] for i in label_columns}
    data['text'] = []
    
    for t in transcripts:
        for line in t:
            
            if re.match('Persuader:', line):
                text, label = process_line(line)
                data['text'].append(text)
                data = update_labels(label_columns, data, label)
                
            else:
                continue
    df = pd.DataFrame(data)
    return df[['text'] + label_columns]

In [132]:
df = generate_dataframe(jsonl, label_columns)

,text,1-RAPPORT,2-NEGOTIATE,3-EMOTION,4-LOGIC,5-AUTHORITY,6-SOCIAL,7-PRESSURE,8-NO-PERSUASION
0,hey!!,1,0,0,0,0,0,0,0
1,a little better what are u doing?,0,0,0,0,0,0,0,1
2,yea i thought u did i just woke and checked t...,0,0,0,0,0,0,0,1
3,so what did u do today?,0,0,0,0,0,0,0,1
4,sounds like fun lol,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
622,:-O,0,0,0,0,0,0,0,1
623,nothing,0,0,0,0,0,0,0,1
624,I would have rather stayed home,0,0,1,0,0,0,0,0
625,lol,0,0,0,0,0,0,0,1
